In [1]:
from queue import PriorityQueue
from collections import deque

In [2]:
initial_state = (3, 3, 0)
goal_state = (0, 0, 1)

In [3]:
def is_valid_state(state):
    priests_left, cannibals_left, boat_position = state
    priests_right = 3 - priests_left
    cannibals_right = 3 - cannibals_left

    if (priests_left < cannibals_left and priests_left > 0) or (priests_right < cannibals_right and priests_right > 0):
        return False

    return True

In [4]:
def get_successors(state):
    successors = []

    for move in [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]:
        delta_p, delta_c = move
        new_state = (
            state[0] - delta_p * state[2],
            state[1] - delta_c * state[2],
            1 - state[2]
        )

        if 0 <= new_state[0] <= 3 and 0 <= new_state[1] <= 3 and is_valid_state(new_state):
            successors.append(new_state)

    return successors

In [5]:
def h(state):
    return state[0] + state[1]

In [6]:
def dfs():
    stack = deque([(initial_state, [])])

    while stack:
        current_state, path = stack.pop()

        if current_state == goal_state:
            return True, len(path), path + [current_state], len(path)

        successors = get_successors(current_state)
        for successor in successors:
            stack.append((successor, path + [current_state]))

    return False, 0, [], 0

In [7]:
def bfs():
    queue = deque([(initial_state, [])])

    while queue:
        current_state, path = queue.popleft()

        if current_state == goal_state:
            return True, len(path), path + [current_state], len(path)

        successors = get_successors(current_state)
        for successor in successors:
            queue.append((successor, path + [current_state]))

    return False, 0, [], 0

In [8]:
def ids():
    depth_limit = 0

    while True:
        result, path, expanded_nodes = depth_limited_dfs(initial_state, depth_limit)
        
        if result == "FOUND":
            return True, len(path), path, expanded_nodes
        elif result == "NOT_FOUND":
            depth_limit += 1
        elif result == "LIMIT_EXCEEDED":
            return False, 0, [], 0

In [9]:
def depth_limited_dfs(state, depth_limit):
    return depth_limited_dfs_recursive(state, depth_limit, [], 0)

In [10]:
def depth_limited_dfs_recursive(state, depth_limit, path, expanded_nodes):
    expanded_nodes += 1

    if state == goal_state:
        return "FOUND", path + [state], expanded_nodes

    if depth_limit == 0:
        return "LIMIT_EXCEEDED", [], expanded_nodes

    successors = get_successors(state)
    for successor in successors:
        result, new_path, expanded_nodes = depth_limited_dfs_recursive(successor, depth_limit - 1, path + [state], expanded_nodes)

        if result == "FOUND":
            return "FOUND", new_path, expanded_nodes

    return "NOT_FOUND", [], expanded_nodes

In [11]:
def greedy():
    frontier = PriorityQueue()
    frontier.put((h(initial_state), 0, initial_state, []))

    while not frontier.empty():
        _, cost, current_state, path = frontier.get()

        if current_state == goal_state:
            return True, cost, path + [current_state], frontier.qsize()

        successors = get_successors(current_state)
        successors.sort(key=lambda s: h(s))
        for successor in successors:
            frontier.put((h(successor), cost + 1, successor, path + [current_state]))

    return False, 0, [], 0

In [12]:
def a_star():
    frontier = PriorityQueue()
    frontier.put((0 + h(initial_state), 0, initial_state, []))

    while not frontier.empty():
        _, cost, current_state, path = frontier.get()

        if current_state == goal_state:
            return True, cost, path + [current_state], frontier.qsize()

        successors = get_successors(current_state)
        successors.sort(key=lambda s: h(s))
        for successor in successors:
            new_cost = cost + 1
            frontier.put((new_cost + h(successor), new_cost, successor, path + [current_state]))

    return False, 0, [], 0

In [13]:
def ida_star():
    threshold = h(initial_state)

    while True:
        result, new_threshold, cost, path = depth_limited_search(initial_state, 0, threshold)
        
        if result == "FOUND":
            return True, cost, path, 0
        elif result == "NOT_FOUND":
            return False, 0, [], 0

        threshold = new_threshold

In [14]:
def depth_limited_search(state, cost, threshold):
    f = cost + h(state)

    if f > threshold:
        return "NOT_FOUND", f, 0, []

    if state == goal_state:
        return "FOUND", f, cost, [state]

    min_threshold = float('inf')

    successors = get_successors(state)
    successors.sort(key=lambda s: h(s))
    for successor in successors:
        result, new_threshold, new_cost, new_path = depth_limited_search(successor, cost + 1, threshold)

        if result == "FOUND":
            return "FOUND", f, new_cost, [state] + new_path

        min_threshold = min(min_threshold, new_threshold)

    return "NOT_FOUND", min_threshold, 0, []

In [27]:
# Uncomment the method you want to use
found, cost, path, expanded_nodes = dfs()
# found, cost, path, expanded_nodes = bfs()
# found, cost, path, expanded_nodes = ids()
# found, cost, path, expanded_nodes = greedy()
# found, cost, path, expanded_nodes = a_star()
# found, cost, path, expanded_nodes = ida_star()

In [28]:
if found:
    print("Solution found!")
    print("Cost:", cost)
    print("Path:", path)
    print("Expanded Nodes:", expanded_nodes)
else:
    print("No solution found.")

Solution found!
Cost: 7
Path: [(3, 3, 0), (3, 3, 1), (2, 2, 0), (2, 2, 1), (1, 1, 0), (1, 1, 1), (0, 0, 0), (0, 0, 1)]
Expanded Nodes: 7
